In [1]:
from valx_ie_parser import *

In [2]:
ct_gov_df = pd.read_csv("Valx/data/nsclc/trail_info.csv")
ct_gov_df = ct_gov_df[['nct_id','eligibility_criteria_textblock','eligibility_criteria_minimum_age',
                          'eligibility_criteria_maximum_age','eligibility_criteria_gender']].dropna()

In [3]:
ie_parser = valex_ie_parser()
out,input_ = ie_parser.extract_variables(ct_gov_df,"Valx/data/nsclc/trail_info_out")

[2020-11-26 20:01:11.665918] found a total of 1350 data items


processing 0



KeyboardInterrupt: 